In [4]:
## Maybe you need to install some libraries. You can just discomment the itens below:
#!pip install pandas
#!pip install requests
#!pip install pysqlite3
#!pip install sqlalchemy
#!pip install bs4

In [5]:
#Request to get the data
import requests
#BeautifulSoup to analyze
from bs4 import BeautifulSoup
#Pandas to structure table
import pandas as pd

In [6]:
base_url = "https://play.google.com/store/search"
page_number = 1
app_data = []

while True:
    url = f"{base_url}?c=apps&page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    app_scores = soup.find_all("span", class_="w2kbF")
    app_names = soup.find_all("span", class_="DdYX5")
    app_owners = soup.find_all("span", class_="wMUdtb")
    app_links = soup.find_all("a", class_="Si6A0c Gy4nib")
    
    if len(app_names) == 0:
        break
    
    for score, name, owner, link in zip(app_scores, app_names, app_owners, app_links):
        app_data.append({
            "app_score": score.text.strip(),
            "app_name": name.text.strip(),
            "app_owner": owner.text.strip(),
            "app_links": "https://play.google.com" + link.get("href") if link else ""
        })
    
    page_number += 1
    
        #limit number of pages scraped or it will cost a lot or remove this if statement to scrape everything
    if page_number == 2:
        break
    

# Create a DataFrame from the collected app data
app_data = pd.DataFrame(app_data)

In [7]:
# Access app_links and retrieve descriptions
app_descriptions_text = []

for app_link in app_data['app_links']:
    if app_link:
        response = requests.get(app_link)
        soup = BeautifulSoup(response.content, "html.parser")
        description_div = soup.find("div", class_="bARER")  # Adjust the class based on the actual structure
        
        if description_div:
            description_parts = description_div.find_all("br")
            app_description_parts = []
            for part in description_parts:
                if "Apple TV channels and content may vary by country or region." in str(part.previous_sibling):
                    app_description_parts.append(str(part.previous_sibling))
                    break
                app_description_parts.append(str(part.previous_sibling))
            
            app_description = "\n".join(app_description_parts).strip()
            app_descriptions_text.append(app_description)
        else:
            app_descriptions_text.append("Description not found")

# Add descriptions to the DataFrame
app_data['app_description_text'] = app_descriptions_text


In [8]:
#First two lines
app_data.head(2)

,app_score,app_name,app_owner,app_links,app_description_text
0,4.0,My Baby Unicorn - Pony Care,TutoTOONS,https://play.google.com/store/apps/details?id=...,You got a gift - a super cute virtual unicorn ...
1,3.8,My Baby Unicorn 2,TutoTOONS,https://play.google.com/store/apps/details?id=...,Welcome to the magical fantasy world of My Bab...


In [9]:
# Last two lines
app_data.tail(2)

,app_score,app_name,app_owner,app_links,app_description_text
28,3.8,kawaii unicorn coloring book,Kawaii Apps,https://play.google.com/store/apps/details?id=...,Are you in love with unicorn ? Are you looking...
29,4.0,Magical Unicorn Candy World,winkypinky,https://play.google.com/store/apps/details?id=...,Find your way into this mystical adventure tha...


In [10]:
#Import libraries to save data to sql
import sqlite3
from sqlalchemy import create_engine

ModuleNotFoundError: No module named 'sqlalchemy'

In [ ]:
# Name of the SQLite Database
db_file = "AppStoreData.db"

# Connect to SQLite using sqlalchemy using UTF-8
engine = create_engine(f"sqlite:///{db_file}")
conn = engine.connect()

# Save data_apps dataframe to the PlayStoreData sql Database.
app_data.to_sql("data_apps", conn, if_exists="replace", index=False)

30

In [ ]:
# URL of the whatsapp page with reviews
base_url = "https://play.google.com/store/apps/details?id=com.whatsapp"
page_num = 1
reviews_per_page = 10  # Number of reviews per page

# Create empty lists to store data
rating_values = []
review_dates = []
review_contents = []

In [ ]:
while True:
    # Construct the URL for the current page
    url = f"{base_url}&showAllReviews=true&page={page_num}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract review ratings
    ratings = soup.find_all("span", class_="F7XJmb")
    rating_values.extend([rating.get("data-number") for rating in ratings])

    # Extract review dates
    dates = soup.find_all("span", class_="bp9Aid")
    review_dates.extend([date.get_text() for date in dates])

    # Extract review contents
    reviews = soup.find_all("div", class_="h3YV2d")  # Updated class name
    review_contents.extend([review.get_text() if review else None for review in reviews])

    # Check if there are no more reviews on the page
    if len(ratings) < reviews_per_page:
        break

    # Increment the page number
    page_num += 1

    #limit number of pages scraped or it will cost a lot or remove this if statement to scrape everything
    if page_num == 20:
        break

In [ ]:
# Ensure all lists are of the same length
min_length = min(len(rating_values), len(review_dates), len(review_contents))
rating_values = rating_values[:min_length]
review_dates = review_dates[:min_length]
review_contents = review_contents[:min_length]

# Create a DataFrame to store the data
comments_whatsapp = {
    "Rating": rating_values,
    "Date": review_dates,
    "Review": review_contents
}
comments_whatsapp = pd.DataFrame(comments_whatsapp)

In [ ]:
# First five lines
comments_whatsapp.head(5)

,Rating,Date,Review
0,1,"August 4, 2023",I'm noticing the app continues to increase in ...
1,2,"August 6, 2023","Um, with the last crash in Germany the audio i..."
2,3,"August 14, 2023",I love this app. To communicate with family wh...
3,4,"August 4, 2023",I'm noticing the app continues to increase in ...
4,5,"August 6, 2023","Um, with the last crash in Germany the audio i..."


In [ ]:
# Save comments_whatsapp dataframe to the PlayStoreData sql database.
comments_whatsapp.to_sql("comments_whatsapp", conn, if_exists="replace", index=False)

57

In [ ]:
# Build SQL query to show data_apps table
query = f'''
SELECT * FROM data_apps limit 5;
'''

# Execute SQL query
resultado_df = pd.read_sql_query(query, conn)

#Show SQL data_app
resultado_df

,app_score,app_name,app_owner,app_links,app_description_text
0,4.0,My Baby Unicorn - Pony Care,TutoTOONS,https://play.google.com/store/apps/details?id=...,You got a gift - a super cute virtual unicorn ...
1,3.8,My Baby Unicorn 2,TutoTOONS,https://play.google.com/store/apps/details?id=...,Welcome to the magical fantasy world of My Bab...
2,4.2,Twinkle - Unicorn Cat Princess,TutoTOONS,https://play.google.com/store/apps/details?id=...,Play the cutest virtual cat care game for girl...
3,4.0,Kpopsies - Hatch Baby Unicorns,TutoTOONS,https://play.google.com/store/apps/details?id=...,<b>Collect Virtual Pets</b>\n<br/>\nMeet Kpops...
4,4.1,My Unicorn dress up for kids,Pazu Games,https://play.google.com/store/apps/details?id=...,"My Unicorn is all about magical colors, cutene..."


In [ ]:
# Build SQL query to show comments_whatsapp table
query = f'''
SELECT * FROM comments_whatsapp limit 5;
'''

# Execute SQL query
resultado_df = pd.read_sql_query(query, conn)

#Show SQL data_app
resultado_df

,Rating,Date,Review
0,1,"August 4, 2023",I'm noticing the app continues to increase in ...
1,2,"August 6, 2023","Um, with the last crash in Germany the audio i..."
2,3,"August 14, 2023",I love this app. To communicate with family wh...
3,4,"August 4, 2023",I'm noticing the app continues to increase in ...
4,5,"August 6, 2023","Um, with the last crash in Germany the audio i..."


In [ ]:
#important
#Close SQLite connection
conn.close()